In [ ]:
!pip3 install transformers==4.15.0 timm==0.4.12 fairscale==0.4.4
!git clone https://github.com/salesforce/BLIP
%cd BLIP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 34.4 MB/s 
     |████████████████████████████████| 376 kB 67.9 MB/s 
     |████████████████████████████████| 235 kB 75.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 51.2 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
     |████████████████████████████████| 880 kB 51.7 MB/s 
     |████████████████████████████████| 3.3 MB 60.9 MB/s 
  Created wheel for fairscale: filename=fairscale-0.4.4-py3-none-any.whl size=292864 sha256=a6f84f382addbece303d3078ea3ccf4381038f59be28a863d58af8aa563b0f1b
  Stored in directory: /root/.cache/pip/wheels/7e/01/9d/ea7ac6a01eb4044323d028f92fe2ac26c36157ad01b2b907c1
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3

In [ ]:
import gdown

In [ ]:
!gdown 1xnBraiIx9vsVAfts8MWJy8CsEtVROKAp

Downloading...
From: https://drive.google.com/uc?id=1xnBraiIx9vsVAfts8MWJy8CsEtVROKAp
To: /content/BLIP/data.tar
100% 12.4G/12.4G [01:46<00:00, 117MB/s]


In [ ]:
!tar -xf data.tar -C dataset/

Testing on one image

In [ ]:
from PIL import Image
import requests
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_demo_image(image_size,device):
    img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
    raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')   

    w,h = raw_image.size
    #display(raw_image.resize((w//5,h//5)))
    
    transform = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 
    image = transform(raw_image).unsqueeze(0).to(device)   
    return image

In [ ]:
from models.blip import blip_decoder

image_size = 384
image = load_demo_image(image_size=image_size, device=device)

model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth'    
model = blip_decoder(pretrained=model_url, image_size=image_size, vit='base')
model.eval()
model = model.to(device)

with torch.no_grad():
    # beam search
    #caption = model.generate(image, sample=False, num_beams=3, max_length=20, min_length=5) 
    # nucleus sampling
    caption = model.generate(image, sample=True, top_p=0.9, max_length=20, min_length=5) 
    print('caption: ')
    print(caption)

reshape position embedding from 196 to 576
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth
caption: 
['a dog sitting on the sand near a girl']


Running the model on dataset

In [ ]:
# Demo for multiple images 
import requests
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
from models.blip import blip_decoder
import os
import glob
import sys


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_demo_image_2(image_size,device, input_image):
   
    raw_image = Image.open(input_image)
    w,h = raw_image.size
    #display(raw_image.resize((w//5,h//5)))
    
    transform = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    
    image = transform(raw_image).unsqueeze(0).to(device)   
    return image

In [ ]:
import json
import os
filenames_train = []
filenames_test = []
with open('annotations_file_2750_v4.json','r') as fp:
	data_train = json.load(fp)
with open('annotations_test_set_v3.json','r') as fp:
	data_test = json.load(fp)

for img in data_train:
  filenames_train.append(img)
for img in data_test:
  filenames_test.append(img.strip())
print(filenames_test[0])
folder_train = "/content/BLIP/dataset/data/train_images"
folder_test = "/content/BLIP/dataset/data/test_images"

filenames = []
for img in filenames_test:
  for dir in os.listdir(folder_train):
    if os.path.exists(folder_train+'/'+dir+'/'+img):
      filenames.append(folder_train+'/'+dir+'/'+img)
      break


for img in filenames_train:
  for dir in os.listdir(folder_train):
    if os.path.exists(folder_train+'/'+dir+'/'+img):
      filenames.append(folder_train+'/'+dir+'/'+img)
      break


print(filenames[0],len(filenames))




1506.jpg
/content/BLIP/dataset/data/train_images/6/1506.jpg 3000


In [ ]:
# with beam search
image_size = 384
img_caption_dict = {}
model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth'
model = blip_decoder(pretrained=model_url, image_size=image_size, vit='large')

def load_images_from_folder(folder):
	images_name = []
	for filename in os.listdir(folder):
		#print(filename)
		images_name.append(folder+'/'+str(filename))
	return images_name

#filenames = glob.glob("/content/BLIP/BLIP_image_caption_demo/karpathy_test_images/*.jpg")
# filenames = []
# for i in range(11):
#   filenames += load_images_from_folder("dataset/data/train_images/"+str(i))
# for i in range(11):
#   filenames += load_images_from_folder("/content/BLIP/dataset/data/test_images/"+str(i))
filenames.sort()
count=0
for image in filenames:
    img = image[:]
    input_image = Image.open(image)
    try:
      image = load_demo_image_2(image_size=image_size, device=device, input_image=image)
    except:
      print("Error : ",image)
      continue
    model.eval()
    model = model.to(device)

    with torch.no_grad():
        #beam search
        caption = model.generate(image, sample=False, num_beams=3, max_length=20, min_length=5)
        #print(caption[0])
        print(count,img)
        img_caption_dict[img] = caption
        count+=1


load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth
0 /content/BLIP/dataset/data/train_images/0/100180.jpg
1 /content/BLIP/dataset/data/train_images/0/101770.jpg
2 /content/BLIP/dataset/data/train_images/0/101910.jpg
3 /content/BLIP/dataset/data/train_images/0/102140.jpg
4 /content/BLIP/dataset/data/train_images/0/102380.jpg
5 /content/BLIP/dataset/data/train_images/0/104270.jpg
6 /content/BLIP/dataset/data/train_images/0/104780.jpg
7 /content/BLIP/dataset/data/train_images/0/105290.jpg
8 /content/BLIP/dataset/data/train_images/0/105700.jpg
9 /content/BLIP/dataset/data/train_images/0/105770.jpg
10 /content/BLIP/dataset/data/train_images/0/106350.jpg
11 /content/BLIP/dataset/data/train_images/0/106640.jpg
12 /content/BLIP/dataset/data/train_images/0/10680.jpg
13 /content/BLIP/dataset/data/train_images/0/107460.jpg
14 /content/BLIP/dataset/data/train_images/0/107930.jpg
15 /content/BLIP/dataset/data/train_images/0/107960.jp

In [ ]:
print(len(img_caption_dict))

2971


In [ ]:
for img in img_caption_dict:
  print(img,img_caption_dict[img])
  break

/content/BLIP/dataset/data/train_images/0/100180.jpg ['a black and white photo of a motorcycle on a road']


In [ ]:
with open("results.json",'w+') as fp:
  json.dump(img_caption_dict,fp,indent=4)

YOLOv5 - Object Detection

In [ ]:
%cd /content

/content


In [ ]:
!python --version

Python 3.7.13


In [ ]:
!git clone https://github.com/ultralytics/yolov5 

Cloning into 'yolov5'...
remote: Enumerating objects: 12440, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 12440 (delta 21), reused 13 (delta 4), pack-reused 12400
Receiving objects: 100% (12440/12440), 12.38 MiB | 14.54 MiB/s, done.
Resolving deltas: 100% (8519/8519), done.


In [ ]:
%cd yolov5

/content/yolov5


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
sys.path.append("/content/yolov5")

In [ ]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'yolov5s') 

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
INFO:yolov5:YOLOv5 🚀 2022-8-3 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

YOLOv5 🚀 2022-8-3 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

INFO:yolov5:Downloading https://github.com/ultralytics/yolov5/releases/download/v6.1/yolov5s.pt to yolov5s.pt...


  0%|          | 0.00/14.1M [00:00<?, ?B/s]

INFO:yolov5:

INFO:yolov5:Fusing layers... 
Fusing layers... 
INFO:yolov5:YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
INFO:yolov5:Adding AutoShape... 
Adding AutoShape... 


In [ ]:
output = {}
count = 0
for img in filenames:
  results = model(img)
  count+=1
  print(count)
  #results.print()
  #print(results.xyxy[0])
  output[img] = {"regions":[]}
  res = results.pandas().xyxy[0]
  for i in res.index:
    output[img]["regions"].append({"name":res["name"][i],"confidence":res["confidence"][i],"bbox":{"xmin": res["xmin"][i], "ymin": res["ymin"][i], "ymax": res["ymax"][i], "xmax": res["xmax"][i]}})
  
  

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
print(len(output))

2993


In [ ]:
with open("results_yolo.json",'w+') as fp:
  json.dump(output,fp,indent=4)